# Building GPT-2 from Scratch with MAX

This interactive notebook guides you through building GPT-2 step by step. Each section corresponds to a step in the tutorial.

## Setup

First, let's verify MAX is installed and import the necessary libraries.

In [ ]:
!pip install --pre modular \
  --index-url https://dl.modular.com/public/nightly/python/simple/

In [ ]:
# Run this cell to verify MAX is installed and import necessary libraries
try:
    from dataclasses import dataclass
    import math
    import numpy as np

    from max.driver import CPU, Device
    from max.dtype import DType
    from max.experimental import functional as F
    from max.experimental.tensor import Tensor
    from max.graph import Dim, DimLike
    from max.nn.module_v3 import Embedding, Linear, Module, Sequential

    # Verify MAX is working by creating a simple tensor
    test_tensor = Tensor.ones([2, 2], dtype=DType.float32, device=CPU())

    print("✓ MAX is installed and working correctly!")
    print(f"✓ All imports successful")
    print(f"✓ Test tensor created: shape={test_tensor.shape}")
    print("\nYou're ready to start building GPT-2!")

except ImportError as e:
    print("❌ Error: MAX is not installed or not accessible")
    print(f"Error details: {e}")
    print("\nTo fix this:")
    print("1. Install MAX: https://docs.modular.com/max/")
    print("2. Run 'pixi install' from the project root")
    print("3. Start Jupyter in the pixi environment: 'pixi run jupyter lab'")

## Step 01: Model Configuration

Before you can implement GPT-2, you need to define its architecture - the dimensions, layer counts, and structural parameters.

Create `GPT2Config`, a class that holds all the architectural decisions for GPT-2.

### Instructions:

1. Add the `@dataclass` decorator to the class
2. Get the parameter values from the [GPT-2 config.json](https://huggingface.co/openai-community/gpt2/blob/main/config.json)
3. Replace each `None` with the correct value:
   - `vocab_size`: 50,257
   - `n_positions`: 1,024
   - `n_embd`: 768
   - `n_layer`: 12
   - `n_head`: 12
   - `n_inner`: 3,072
   - `layer_norm_epsilon`: 1e-5

In [ ]:
# TODO: Add @dataclass decorator
class GPT2Config:
    vocab_size: int = None  # TODO: Replace with correct value
    n_positions: int = None  # TODO: Replace with correct value
    n_embd: int = None  # TODO: Replace with correct value
    n_layer: int = None  # TODO: Replace with correct value
    n_head: int = None  # TODO: Replace with correct value
    n_inner: int = None  # TODO: Replace with correct value
    layer_norm_epsilon: float = None  # TODO: Replace with correct value

In [ ]:
# Validation cell - run this to check your configuration
config = GPT2Config()
assert config.vocab_size == 50257, f"Expected vocab_size=50257, got {config.vocab_size}"
assert config.n_positions == 1024, f"Expected n_positions=1024, got {config.n_positions}"
assert config.n_embd == 768, f"Expected n_embd=768, got {config.n_embd}"
assert config.n_layer == 12, f"Expected n_layer=12, got {config.n_layer}"
assert config.n_head == 12, f"Expected n_head=12, got {config.n_head}"
assert config.n_inner == 3072, f"Expected n_inner=3072, got {config.n_inner}"
assert config.layer_norm_epsilon == 1e-5, f"Expected layer_norm_epsilon=1e-5, got {config.layer_norm_epsilon}"
print("✓ Step 01 complete! Configuration is correct.")

## Step 02: Causal Masking

Implement the `causal_mask()` function that prevents the model from "seeing" future tokens during autoregressive generation.

The mask creates a lower triangular pattern where each token can only attend to itself and previous tokens.

### Instructions:

1. Add `@F.functional` decorator to convert the function to a MAX graph operation
2. Calculate total sequence length: `n = Dim(sequence_length) + num_tokens`
3. Create a `-inf` constant: `Tensor.constant(float("-inf"), dtype=dtype, device=device)`
4. Broadcast to target shape: `F.broadcast_to(mask, shape=(sequence_length, n))`
5. Apply band_part to create lower triangular pattern: `F.band_part(mask, num_lower=None, num_upper=0, exclude=True)`

In [ ]:
# TODO: Add @F.functional decorator
def causal_mask(
    sequence_length: DimLike,
    num_tokens: DimLike,
    dtype: DType,
    device: Device,
) -> Tensor:
    # TODO: Calculate total length n
    n = None

    # TODO: Create -inf constant tensor
    mask = None

    # TODO: Broadcast to (sequence_length, n) shape
    mask = None

    # TODO: Apply band_part to create lower triangular pattern
    return None

In [ ]:
# Validation cell
test_mask = causal_mask(3, 0, DType.float32, CPU())
print("Causal mask for sequence length 3:")
print(test_mask)
print("\n✓ Step 02 complete! Causal mask is working.")

## Step 03: Layer Normalization

Create the `LayerNorm` class that normalizes activations across the feature dimension to stabilize training.

### Instructions:

1. Initialize `self.weight` using `Tensor.ones([dim])`
2. Initialize `self.bias` using `Tensor.zeros([dim])`
3. Apply layer normalization using `F.layer_norm(x, gamma=self.weight, beta=self.bias, epsilon=self.eps)`

In [ ]:
class LayerNorm(Module):
    def __init__(self, dim: int, eps: float = 1e-5):
        super().__init__()
        self.eps = eps
        # TODO: Initialize weight parameter with ones
        self.weight = None
        # TODO: Initialize bias parameter with zeros
        self.bias = None

    def forward(self, x: Tensor) -> Tensor:
        # TODO: Apply layer normalization
        return None

In [ ]:
# Validation cell
ln = LayerNorm(768)
test_input = Tensor.randn([1, 10, 768], dtype=DType.float32, device=CPU())
output = ln(test_input)
print(f"Input shape: {test_input.shape}")
print(f"Output shape: {output.shape}")
print("✓ Step 03 complete! Layer normalization is working.")

## Step 04: Feed-Forward Network (MLP)

Build the `MLP` class, a two-layer feed-forward network with GELU activation.

### Instructions:

1. Create expansion layer: `Linear(embed_dim, intermediate_size, bias=True)` → `self.c_fc`
2. Create projection layer: `Linear(intermediate_size, embed_dim, bias=True)` → `self.c_proj`
3. In forward pass:
   - Apply expansion: `self.c_fc(hidden_states)`
   - Apply GELU: `F.gelu(hidden_states, approximate="tanh")`
   - Apply projection: `self.c_proj(hidden_states)`

In [ ]:
class MLP(Module):
    def __init__(self, embed_dim: int, intermediate_size: int):
        super().__init__()
        # TODO: Create expansion layer (c_fc)
        self.c_fc = None
        # TODO: Create projection layer (c_proj)
        self.c_proj = None

    def forward(self, hidden_states: Tensor) -> Tensor:
        # TODO: Apply expansion
        hidden_states = None
        # TODO: Apply GELU activation
        hidden_states = None
        # TODO: Apply projection and return
        return None

In [ ]:
# Validation cell
mlp = MLP(768, 3072)
test_input = Tensor.randn([1, 10, 768], dtype=DType.float32, device=CPU())
output = mlp(test_input)
print(f"Input shape: {test_input.shape}")
print(f"Output shape: {output.shape}")
print("✓ Step 04 complete! MLP is working.")

## Step 05: Token Embeddings

Create the token embedding layer that converts discrete token IDs into continuous vector representations.

### Instructions:

1. Create token embedding layer: `Embedding(config.vocab_size, dim=config.n_embd)` → `self.wte`
2. In forward pass: `self.wte(input_ids)`

In [ ]:
class TokenEmbedding(Module):
    def __init__(self, config: GPT2Config):
        super().__init__()
        # TODO: Create token embedding layer (wte)
        self.wte = None

    def forward(self, input_ids: Tensor) -> Tensor:
        # TODO: Lookup token embeddings
        return None

In [ ]:
# Validation cell
token_emb = TokenEmbedding(config)
test_tokens = Tensor([1, 2, 3, 4, 5], dtype=DType.int64, device=CPU())
output = token_emb(test_tokens)
print(f"Input tokens shape: {test_tokens.shape}")
print(f"Output embeddings shape: {output.shape}")
print("✓ Step 05 complete! Token embeddings working.")

## Step 06: Position Embeddings

Create position embeddings to encode where each token appears in the sequence.

### Instructions:

1. Create position embedding layer: `Embedding(config.n_positions, dim=config.n_embd)` → `self.wpe`
2. In forward pass: `self.wpe(position_ids)`

In [ ]:
class PositionEmbedding(Module):
    def __init__(self, config: GPT2Config):
        super().__init__()
        # TODO: Create position embedding layer (wpe)
        self.wpe = None

    def forward(self, position_ids: Tensor) -> Tensor:
        # TODO: Lookup position embeddings
        return None

In [ ]:
# Validation cell
pos_emb = PositionEmbedding(config)
test_positions = Tensor.arange(5, dtype=DType.int64, device=CPU())
output = pos_emb(test_positions)
print(f"Input positions shape: {test_positions.shape}")
print(f"Output embeddings shape: {output.shape}")
print("✓ Step 06 complete! Position embeddings working.")

## Step 07: Q/K/V Projections (Single Head)

Implement Q/K/V projections for attention. GPT-2 uses a combined projection layer for efficiency.

### Instructions:

1. Create combined Q/K/V projection: `Linear(config.n_embd, 3 * config.n_embd, bias=True)` → `self.c_attn`
2. In forward pass:
   - Apply projection: `qkv = self.c_attn(x)`
   - Split into Q, K, V: `F.split(qkv, [self.n_embd, self.n_embd, self.n_embd], axis=-1)`

In [ ]:
class QKVProjection(Module):
    def __init__(self, config: GPT2Config):
        super().__init__()
        self.n_embd = config.n_embd
        # TODO: Create combined Q/K/V projection layer (c_attn)
        self.c_attn = None

    def forward(self, x: Tensor) -> tuple[Tensor, Tensor, Tensor]:
        # TODO: Apply combined projection
        qkv = None
        # TODO: Split into Q, K, V
        query, key, value = None, None, None
        return query, key, value

In [ ]:
# Validation cell
qkv_proj = QKVProjection(config)
test_input = Tensor.randn([1, 10, 768], dtype=DType.float32, device=CPU())
q, k, v = qkv_proj(test_input)
print(f"Input shape: {test_input.shape}")
print(f"Query shape: {q.shape}")
print(f"Key shape: {k.shape}")
print(f"Value shape: {v.shape}")
print("✓ Step 07 complete! Q/K/V projections working.")

## Step 08: Attention Mechanism with Causal Masking

Implement the core attention mechanism using scaled dot-product attention.

### Instructions:

1. Compute attention scores: `query @ key.transpose(-1, -2)`
2. Scale scores: divide by `math.sqrt(int(value.shape[-1]))`
3. Apply causal mask: `attn_weights + causal_mask(...)`
4. Apply softmax: `F.softmax(attn_weights)`
5. Compute output: `attn_weights @ value`

In [ ]:
def compute_attention(query: Tensor, key: Tensor, value: Tensor) -> Tensor:
    # TODO: Compute attention scores (Q @ K^T)
    attn_weights = None

    # TODO: Scale by sqrt(d_k)
    scale_factor = None
    attn_weights = None

    # TODO: Apply causal mask
    seq_len = query.shape[-2]
    mask = causal_mask(seq_len, 0, dtype=query.dtype, device=query.device)
    attn_weights = None

    # TODO: Apply softmax
    attn_weights = None

    # TODO: Compute weighted sum of values
    return None

In [ ]:
# Validation cell
test_q = Tensor.randn([1, 10, 768], dtype=DType.float32, device=CPU())
test_k = Tensor.randn([1, 10, 768], dtype=DType.float32, device=CPU())
test_v = Tensor.randn([1, 10, 768], dtype=DType.float32, device=CPU())
output = compute_attention(test_q, test_k, test_v)
print(f"Query shape: {test_q.shape}")
print(f"Output shape: {output.shape}")
print("✓ Step 08 complete! Attention mechanism working.")

## Step 09: Multi-Head Attention

Extend single-head attention to multi-head attention by splitting embeddings across 12 heads.

### Instructions:

1. Create Q/K/V projection: `Linear(config.n_embd, 3 * config.n_embd, bias=True)` → `self.c_attn`
2. Create output projection: `Linear(config.n_embd, config.n_embd, bias=True)` → `self.c_proj`
3. Implement `_split_heads`: reshape and transpose to `[batch, num_heads, seq_length, head_dim]`
4. Implement `_merge_heads`: reverse the splitting operation
5. In forward: project → split heads → attention → merge heads → output projection

In [ ]:
class MultiHeadAttention(Module):
    def __init__(self, config: GPT2Config):
        super().__init__()
        self.num_heads = config.n_head
        self.head_dim = config.n_embd // config.n_head
        self.split_size = config.n_embd

        # TODO: Create combined Q/K/V projection
        self.c_attn = None
        # TODO: Create output projection
        self.c_proj = None

    def _split_heads(self, tensor: Tensor, num_heads: int, attn_head_size: int) -> Tensor:
        # TODO: Reshape to add head dimension
        new_shape = tensor.shape[:-1] + [num_heads, attn_head_size]
        tensor = None
        # TODO: Transpose to move heads to position 1
        return None

    def _merge_heads(self, tensor: Tensor, num_heads: int, attn_head_size: int) -> Tensor:
        # TODO: Transpose heads back
        tensor = None
        # TODO: Reshape to flatten heads
        new_shape = tensor.shape[:-2] + [num_heads * attn_head_size]
        return None

    def forward(self, hidden_states: Tensor) -> Tensor:
        # TODO: Project to Q/K/V and split
        qkv = None
        query, key, value = None, None, None

        # TODO: Split heads for Q, K, V
        query = None
        key = None
        value = None

        # TODO: Compute attention
        attn_output = None

        # TODO: Merge heads
        attn_output = None

        # TODO: Apply output projection
        return None

In [ ]:
# Validation cell
mha = MultiHeadAttention(config)
test_input = Tensor.randn([1, 10, 768], dtype=DType.float32, device=CPU())
output = mha(test_input)
print(f"Input shape: {test_input.shape}")
print(f"Output shape: {output.shape}")
print("✓ Step 09 complete! Multi-head attention working.")

## Step 10: Residual Connections and Layer Normalization Pattern

The LayerNorm class is already implemented in Step 03. This step demonstrates the pre-norm pattern used in GPT-2.

The pattern is: `x = x + sublayer(layer_norm(x))`

In [ ]:
# Demonstration of pre-norm pattern
def apply_with_residual(x: Tensor, layer_norm: LayerNorm, sublayer: Module) -> Tensor:
    """Apply pre-norm pattern: x = x + sublayer(layer_norm(x))"""
    return x + sublayer(layer_norm(x))

# Validation
ln = LayerNorm(768)
mlp_layer = MLP(768, 3072)
test_input = Tensor.randn([1, 10, 768], dtype=DType.float32, device=CPU())
output = apply_with_residual(test_input, ln, mlp_layer)
print(f"Input shape: {test_input.shape}")
print(f"Output shape: {output.shape}")
print("✓ Step 10 complete! Pre-norm pattern demonstrated.")

## Step 11: Transformer Block

Combine attention, MLP, layer normalization, and residual connections into a complete transformer block.

### Instructions:

1. Create first layer norm: `LayerNorm(config.n_embd, eps=config.layer_norm_epsilon)` → `self.ln_1`
2. Create attention: `MultiHeadAttention(config)` → `self.attn`
3. Create second layer norm: `LayerNorm(config.n_embd, eps=config.layer_norm_epsilon)` → `self.ln_2`
4. Create MLP: `MLP(config.n_embd, config.n_inner)` → `self.mlp`
5. In forward:
   - `x = x + self.attn(self.ln_1(x))`
   - `x = x + self.mlp(self.ln_2(x))`

In [ ]:
class TransformerBlock(Module):
    def __init__(self, config: GPT2Config):
        super().__init__()
        # TODO: Create first layer norm (ln_1)
        self.ln_1 = None
        # TODO: Create multi-head attention (attn)
        self.attn = None
        # TODO: Create second layer norm (ln_2)
        self.ln_2 = None
        # TODO: Create MLP (mlp)
        self.mlp = None

    def forward(self, x: Tensor) -> Tensor:
        # TODO: Apply attention with pre-norm and residual
        x = None
        # TODO: Apply MLP with pre-norm and residual
        x = None
        return x

In [ ]:
# Validation cell
block = TransformerBlock(config)
test_input = Tensor.randn([1, 10, 768], dtype=DType.float32, device=CPU())
output = block(test_input)
print(f"Input shape: {test_input.shape}")
print(f"Output shape: {output.shape}")
print("✓ Step 11 complete! Transformer block working.")

## Step 12: Stacking Transformer Blocks

Stack 12 transformer blocks with embeddings and final normalization to create the complete GPT-2 model.

### Instructions:

1. Create token embeddings: `Embedding(config.vocab_size, dim=config.n_embd)` → `self.wte`
2. Create position embeddings: `Embedding(config.n_positions, dim=config.n_embd)` → `self.wpe`
3. Stack transformer blocks: `Sequential(*[TransformerBlock(config) for _ in range(config.n_layer)])` → `self.h`
4. Create final layer norm: `LayerNorm(config.n_embd, eps=config.layer_norm_epsilon)` → `self.ln_f`
5. In forward:
   - Get sequence length and create positions
   - Combine token and position embeddings
   - Pass through transformer blocks
   - Apply final layer norm

In [ ]:
class GPT2Model(Module):
    def __init__(self, config: GPT2Config):
        super().__init__()
        # TODO: Create token embeddings (wte)
        self.wte = None
        # TODO: Create position embeddings (wpe)
        self.wpe = None
        # TODO: Stack transformer blocks (h)
        self.h = None
        # TODO: Create final layer norm (ln_f)
        self.ln_f = None

    def forward(self, input_ids: Tensor) -> Tensor:
        # TODO: Get sequence length
        seq_length = None

        # TODO: Create position indices
        positions = None

        # TODO: Get token and position embeddings
        token_embeds = None
        pos_embeds = None

        # TODO: Combine embeddings
        hidden_states = None

        # TODO: Pass through transformer blocks
        hidden_states = None

        # TODO: Apply final layer norm
        return None

In [ ]:
# Validation cell
model = GPT2Model(config)
test_tokens = Tensor([[1, 2, 3, 4, 5]], dtype=DType.int64, device=CPU())
output = model(test_tokens)
print(f"Input tokens shape: {test_tokens.shape}")
print(f"Output hidden states shape: {output.shape}")
print("✓ Step 12 complete! GPT-2 model working.")

## Step 13: Language Model Head

Add the final linear projection layer that converts hidden states to vocabulary logits.

### Instructions:

1. Create transformer: `GPT2Model(config)` → `self.transformer`
2. Create LM head: `Linear(config.n_embd, config.vocab_size, bias=False)` → `self.lm_head`
3. In forward:
   - Get hidden states from transformer
   - Project to logits with LM head

In [ ]:
class GPT2LMHeadModel(Module):
    def __init__(self, config: GPT2Config):
        super().__init__()
        # TODO: Create transformer
        self.transformer = None
        # TODO: Create LM head (note: bias=False)
        self.lm_head = None

    def forward(self, input_ids: Tensor) -> Tensor:
        # TODO: Get hidden states from transformer
        hidden_states = None
        # TODO: Project to vocabulary logits
        logits = None
        return logits

In [ ]:
# Validation cell
lm_model = GPT2LMHeadModel(config)
test_tokens = Tensor([[1, 2, 3, 4, 5]], dtype=DType.int64, device=CPU())
logits = lm_model(test_tokens)
print(f"Input tokens shape: {test_tokens.shape}")
print(f"Output logits shape: {logits.shape}")
print(f"Vocabulary size: {config.vocab_size}")
print("✓ Step 13 complete! Language model head working.")

## Step 14: Text Generation

Implement text generation with temperature and sampling.

### Instructions:

1. Implement `generate_next_token`:
   - Get logits from model
   - Extract last position logits
   - Apply temperature scaling
   - Sample from probability distribution or use greedy decoding
2. Implement `generate`:
   - Loop for max_new_tokens iterations
   - Generate next token
   - Concatenate to sequence

In [ ]:
def generate_next_token(
    model: GPT2LMHeadModel,
    input_ids: Tensor,
    temperature: float = 1.0,
    do_sample: bool = True,
) -> Tensor:
    # TODO: Get logits from model
    logits = None

    # TODO: Extract last position logits
    next_token_logits = None

    if do_sample:
        # TODO: Create temperature tensor
        temp_tensor = None
        # TODO: Scale logits by temperature
        next_token_logits = None

        # TODO: Convert to probabilities
        probs = None

        # TODO: Transfer to CPU and convert to NumPy
        probs_np = None

        # TODO: Sample from distribution
        next_token_id = None

        # TODO: Convert back to tensor
        next_token_tensor = None
    else:
        # TODO: Greedy decoding (argmax)
        next_token_tensor = None

    return next_token_tensor

def generate(
    model: GPT2LMHeadModel,
    input_ids: Tensor,
    max_new_tokens: int,
    temperature: float = 1.0,
    do_sample: bool = True,
) -> Tensor:
    # TODO: Initialize with input tokens
    generated_tokens = input_ids

    # TODO: Generate loop
    for _ in range(max_new_tokens):
        # TODO: Generate next token
        next_token = None

        # TODO: Reshape to 2D
        next_token_2d = None

        # TODO: Concatenate to sequence
        generated_tokens = None

    return generated_tokens

In [ ]:
# Validation cell
# Note: This will generate random tokens since we haven't loaded pretrained weights
prompt = Tensor([[15496, 995]], dtype=DType.int64, device=CPU())  # "Hello world"
generated = generate(lm_model, prompt, max_new_tokens=5, temperature=1.0, do_sample=False)
print(f"Prompt shape: {prompt.shape}")
print(f"Generated sequence shape: {generated.shape}")
print(f"Generated token IDs: {generated}")
print("\n✓ Step 14 complete! Text generation working.")
print("\n🎉 Congratulations! You've built GPT-2 from scratch!")

## Summary

You've completed all 14 steps and built a complete GPT-2 model from scratch using MAX!

### What you've built:

1. **Model configuration** - Architecture parameters
2. **Causal masking** - Prevents attending to future tokens
3. **Layer normalization** - Stabilizes training
4. **Feed-forward network (MLP)** - Adds non-linearity
5. **Token embeddings** - Converts token IDs to vectors
6. **Position embeddings** - Encodes sequence order
7. **Q/K/V projections** - Prepares inputs for attention
8. **Attention mechanism** - Core attention computation
9. **Multi-head attention** - Parallel attention heads
10. **Residual connections** - Enables deep networks
11. **Transformer block** - Complete repeating unit
12. **Model stacking** - Combines all components
13. **Language model head** - Projects to vocabulary
14. **Text generation** - Autoregressive sampling
